In [474]:
from selenium import webdriver
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [487]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

In [510]:
driver.get('https://www.google.com/maps/search/restaurantes/@4.761191,-74.0557006,19z/data=!4m2!2m1!6e5?hl=es&entry=ttu')

In [514]:
def extract_data_restaurants(restaurants):
    
    scroll=driver.find_element(By.CLASS_NAME,'Nv2PK')
    iframe=driver.find_element(By.CLASS_NAME,'Nv2PK')

    scroll_origin = ScrollOrigin.from_element(iframe)

    for i in range(0,5):
        ActionChains(driver)\
            .move_to_element(scroll)\
            .scroll_from_origin(scroll_origin,0, 50000)\
            .perform()
    
    maps_results = driver.find_elements(By.CLASS_NAME,'Nv2PK')
    print(len(maps_results))
    for result in maps_results:
        restaurant = {}
        try:
            name = result.find_element(By.CLASS_NAME, 'qBF1Pd') 
            restaurant['name'] = name.text
        except NoSuchElementException:
            restaurant['name'] = None
            
        try:  
            rating = result.find_element(By.CLASS_NAME, 'MW4etd') 
            restaurant['rating'] = rating.text
        except NoSuchElementException:
            restaurant['rating'] = None
            
        try:
            number_commnets_google = result.find_element(By.CLASS_NAME, 'UY7F9') 
            restaurant['number_commnets_google'] = int(number_commnets_google.text.replace("(", "").replace(")", "").replace(".", ""))
        except NoSuchElementException:
            restaurant['number_commnets_google'] = None
            
        try:
            address = result.find_elements(By.CLASS_NAME, 'W4Efsd') 
            line = address[2].text.split('·')
            
            if len(line)>1:
                restaurant['address'] = line[1]
                restaurant['type'] = line[0]
            elif len(line)== 1:
                restaurant['type'] = line[0]
                
        except NoSuchElementException:
            restaurant['address'] = None
            
        try:
            link= result.find_element(By.CLASS_NAME, 'hfpxzc')
            restaurant['google_maps_url'] = link.get_attribute('href')
        except NoSuchElementException:
            restaurant['address'] = None
        
        restaurants.append(restaurant)
    return restaurants

In [515]:
def navigate_map(lat, lon, restaurants):
    driver.get('https://www.google.com/maps/search/restaurantes/@'+str(lat)+','+str(lon)+',19z/data=!4m2!2m1!6e5?hl=es&entry=ttu')
    restaurants=extract_data_restaurants(restaurants)
    return restaurants

In [516]:
lat = 4.6038511
lon = -74.0756006
restaurants = []

for i in range(0,5):
    lat = lat + i*0.001
    lon = -74.0756006
    for j in range(0,5):
        lon = lon + j*0.001
        print(str(lat),'-',str(lon))
        restaurants = navigate_map(lat, lon, restaurants) 
        
print(restaurants)

4.6038511 - -74.0756006
20
4.6038511 - -74.0746006
20
4.6038511 - -74.0726006
20
4.6038511 - -74.0696006
20
4.6038511 - -74.0656006
20
4.6048511 - -74.0756006
20
4.6048511 - -74.0746006
20
4.6048511 - -74.0726006
20
4.6048511 - -74.0696006
20
4.6048511 - -74.0656006
20
4.6068511 - -74.0756006
20
4.6068511 - -74.0746006
20
4.6068511 - -74.0726006
20
4.6068511 - -74.0696006
20
4.6068511 - -74.0656006
20
4.6098511 - -74.0756006
20
4.6098511 - -74.0746006
20
4.6098511 - -74.0726006
20
4.6098511 - -74.0696006
20
4.6098511 - -74.0656006
20
4.6138511 - -74.0756006
20
4.6138511 - -74.0746006
20
4.6138511 - -74.0726006
20
4.6138511 - -74.0696006
20
4.6138511 - -74.0656006
20
[{'name': 'Restaurante Morena Parrilla', 'rating': '4,0', 'number_commnets_google': 17, 'address': ' Cra. 12 # 15-25 Piso 2. Centro Comercial Gran Esquina La', 'type': 'Colombiana ', 'google_maps_url': 'https://www.google.com/maps/place/Restaurante+Morena+Parrilla/data=!4m7!3m6!1s0x8e3f99a082664d3f:0x34fb2db9e6f1274a!8m2!3d

In [519]:
df = pd.DataFrame(restaurants)
print(df.duplicated(keep='first').value_counts())
df=df.drop_duplicates().reset_index().drop(columns=['index'])
df

True     272
False    228
dtype: int64


,name,rating,number_commnets_google,address,type,google_maps_url
0,Restaurante Morena Parrilla,"4,0",17.0,Cra. 12 # 15-25 Piso 2. Centro Comercial Gran...,Colombiana,https://www.google.com/maps/place/Restaurante+...
1,RESTAURANTE GUSTA MAS,"5,0",2.0,Cl. 16 #12-55,Colombiana,https://www.google.com/maps/place/RESTAURANTE+...
2,Restaurante El Antojo de La 15,None,NaN,Cl. 15 #32,Restaurante,https://www.google.com/maps/place/Restaurante+...
3,Restaurante Pescadero Del Centro,"4,5",40.0,Cra. 9 #58,Marisco,https://www.google.com/maps/place/Restaurante+...
4,Restaurante Cafetería Aqúm,None,NaN,NaN,Restaurante,https://www.google.com/maps/place/Restaurante+...
...,...,...,...,...,...,...
223,Anna & Otto Macarena,"4,6",671.0,Cra. 4a #26c-35,Pizza,https://www.google.com/maps/place/Anna+%26+Ott...
224,Pastis Ristorante,"4,4",46.0,Cra. 4a #27 - 86,Italiana,https://www.google.com/maps/place/Pastis+Risto...
225,Restaurante rincón Casa de Paella,"4,5",170.0,Cl. 26c #4a-15,Restaurante,https://www.google.com/maps/place/Restaurante+...
226,Bangkok Thai Cuisine,"4,5",192.0,Cl. 26c #4-41,Tailandesa,https://www.google.com/maps/place/Bangkok+Thai...


In [521]:
df['type'].value_counts()

Restaurante              123
Colombiana                40
Restaurante               21
Italiana                   7
Vegetariana                4
Mexicana                   4
Marisco                    3
Comida rápida              3
Desayuno                   3
China                      3
Peruana                    3
Pizza                      2
Bar de tapas               1
Parrilla                   1
Española                   1
Fusión asiática            1
Caribeña                   1
Serbia                     1
India                      1
Ideal para familias        1
Vegetariana estricta       1
Libanesa                   1
Criolla                    1
Tailandesa                 1
Name: type, dtype: int64